In [1]:
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
# Definiere den Umfang (Scope) für Google Analytics
SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
# Pfad zur JSON-Datei mit den Service-Konto-Schlüsseln
KEY_FILE_LOCATION = '../realtime-api.json'
# Deine Google Analytics View ID
VIEW_ID = '239560925'
# Authentifizierung
credentials = ServiceAccountCredentials.from_json_keyfile_name(
    KEY_FILE_LOCATION, SCOPES)
# API-Client aufbauen
analytics = build('analyticsreporting', 'v4', credentials=credentials)

FileNotFoundError: [Errno 2] No such file or directory: 'realtime-api.json'